# Probabilistic Principal Component Analysis Implementation
Implementation of Probabilistic PCA (ppca) algorithm using EM algorithm

**References**
* Tipping, M. E., & Bishop, C. M. (1999). Probabilistic Principal Component Analysis, 611–622.

***
## Probability model
\begin{align}
y &= \mathbf{W}x + \mu + \epsilon \\
x &\sim \mathcal{N} (0,I) \\
\epsilon &\sim \mathcal{N} (0,\sigma^2I)
\end{align}

where $ \mathbf{W} $ is $d \times q$ matrix. Then the conditional distribution of $y$ over $x$ is given by
\begin{equation}
y|x \sim \mathcal{N} (\mathbf{W}x+\mu, \sigma^2I)
\end{equation}
and marginal distribution of $y$
\begin{equation}
y \sim \mathcal{N} (\mu, C)
\end{equation}
and the observation covariance is specified by $C = WW^\intercal +\sigma^2I$.

***
## EM for PPCA
The complete log-likelihood of the model, including both observation $\mathbf{Y} = \{y_1,y_2,\cdots,y_n\}$ and latent variable $\mathbf{X} = \{x_1,x_2,\cdots,x_n\}$, is
\begin{align}
\mathcal{L} = \log p(\mathbf{Y},\mathbf{Z}|\mu, \mathbf{W}, \sigma^2) 
\end{align}

***
## Initialization

In [2]:
import numpy as np